## Assignment 1

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving UIT-VSFC.zip to UIT-VSFC.zip


In [ ]:
import zipfile
import io
with zipfile.ZipFile(io.BytesIO(uploaded['UIT-VSFC.zip']), 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
!pip3 install py_vncorenlp

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.8 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4304 sha256=cc283b045778a195c54fcadff7fe365fa263f1eefff29d4b415ddae6610c9573
  Stored in directory: /root/.cache/pip/wheels/6d/2d/d6/158260bfd6820d144535857b80cc112bee5c3aa6d81b6dc049
Successfully built py_vncorenlp


In [ ]:
!mkdir vncorenlp

In [ ]:
import py_vncorenlp

py_vncorenlp.download_model(save_dir='/content/vncorenlp')
annotator = py_vncorenlp.VnCoreNLP(save_dir='/content/vncorenlp')

In [ ]:
import json

def load_json_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

In [ ]:
train_data = load_json_data('/content/UIT-VSFC/UIT-VSFC-train.json')
train_data[0]

{'sentence': 'slide giáo trình đầy đủ .',
 'sentiment': 'positive',
 'topic': 'training_program'}

In [ ]:
def word_segment(data):
    for item in data:
        sentence = item['sentence']
        segmented = annotator.annotate_text(sentence)[0]
        annotated = []
        for word in segmented:
            wordForm = word['wordForm']
            annotated.append(wordForm)
        annotated = ' '.join(annotated)
        item['segmented'] = annotated
    return data
train_data = word_segment(train_data)
train_data[0]
test_data = word_segment(load_json_data('/content/UIT-VSFC/UIT-VSFC-test.json'))

## Assignment 2

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [ ]:
import numpy as np
np.random.seed(42)

In [ ]:
def sentence_embedding(sentence):
    input_ids = tokenizer.encode(sentence, return_tensors='pt')
    with torch.no_grad():
        last_hidden_state = phobert(input_ids)[0]
        return last_hidden_state[0][1:-1].mean(dim=0).numpy()

In [ ]:

def dataset_embedded(data):
    embeddings, labels = [], []
    for item in data:
        embedded = sentence_embedding(item['segmented'])
        embeddings.append(embedded)
        labels.append(item['sentiment'])
    return np.array(embeddings), np.array(labels)

In [ ]:
X_Train, y_Train = dataset_embedded(train_data)
X_Test, y_Test = dataset_embedded(test_data)

## Assignment 3

In [ ]:
class GaussianMixtureModel:
    def __init__(self, n_components=3, max_iter=100, tol=1e-4, reg_covar=1e-3):
        """
        Khởi tạo mô hình GMM với số thành phần, số vòng lặp, ngưỡng hội tụ và hệ số regularization.
        """
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
        self.reg_covar = reg_covar

    def fit(self, X, verbose=False):
        """
        Huấn luyện mô hình GMM trên dữ liệu X bằng thuật toán Expectation-Maximization (EM).
        """
        self._initialize_parameters(X)
        prev_ll = -np.inf
        self.log_likelihoods = []

        for iteration in range(self.max_iter):
            # Bước E: Tính ma trận trách nhiệm
            self._e_step(X)

            # Bước M: Cập nhật các tham số
            self._m_step(X)

            # Tính log-likelihood
            ll = self._compute_log_likelihood(X)
            self.log_likelihoods.append(ll)

            if verbose:
                print(f"Iter {iteration}, log-likelihood: {ll:.6f}")

            if not np.isfinite(ll):
                print("⛔ Dừng sớm: Log-likelihood không ổn định (NaN hoặc Inf).")
                break

            if abs(ll - prev_ll) < self.tol:
                if verbose:
                    print("✅ Hội tụ.")
                break

            prev_ll = ll

    def _initialize_parameters(self, X):
        """
        Khởi tạo trọng số, trung bình và hiệp phương sai ban đầu.
        """
        num_samples, num_features = X.shape
        self.weights = np.full(self.n_components, 1 / self.n_components)
        self.means = X[np.random.choice(num_samples, self.n_components, replace=False)]
        base_cov = np.cov(X, rowvar=False) + self.reg_covar * np.eye(num_features)
        self.covs = np.array([base_cov.copy() for _ in range(self.n_components)])

    def _e_step(self, X):
        """
        Bước E: Tính xác suất trách nhiệm (posterior) cho mỗi điểm dữ liệu và mỗi thành phần Gaussian.
        """
        num_samples = X.shape[0]
        self.resp = np.zeros((num_samples, self.n_components))

        for k in range(self.n_components):
            self.resp[:, k] = self.weights[k] * self._multivariate_normal(X, self.means[k], self.covs[k])

        # Chuẩn hóa theo hàng để mỗi hàng là một phân phối xác suất
        row_sums = np.clip(self.resp.sum(axis=1, keepdims=True), 1e-12, None)
        self.resp /= row_sums

    def _m_step(self, X):
        """
        Bước M: Cập nhật trọng số, trung bình và hiệp phương sai từ ma trận trách nhiệm.
        """
        num_samples, num_features = X.shape
        component_sums = self.resp.sum(axis=0)  # (K,)
        self.weights = component_sums / num_samples
        self.means = (self.resp.T @ X) / component_sums[:, None]

        for k in range(self.n_components):
            x_centered = X - self.means[k]
            weighted_diffs = self.resp[:, k][:, None] * x_centered
            cov_k = (weighted_diffs.T @ x_centered) / component_sums[k]
            self.covs[k] = cov_k + self.reg_covar * np.eye(num_features)

    def _compute_log_likelihood(self, X):
        """
        Tính tổng log-likelihood trên toàn bộ tập dữ liệu.
        """
        total_prob = np.zeros(X.shape[0])
        for k in range(self.n_components):
            total_prob += self.weights[k] * self._multivariate_normal(X, self.means[k], self.covs[k])
        return np.sum(np.log(total_prob + 1e-12))

    def _multivariate_normal(self, X, mean, cov):
        """
        Tính mật độ xác suất Gaussian đa biến.
        """
        num_features = X.shape[1]
        x_centered = X - mean

        try:
            inv_cov = np.linalg.inv(cov)
        except np.linalg.LinAlgError:
            cov += self.reg_covar * np.eye(num_features)
            inv_cov = np.linalg.inv(cov)

        det_cov = max(np.linalg.det(cov), 1e-12)
        exponent = np.einsum('ij,jk,ik->i', x_centered, inv_cov, x_centered)
        norm_const = np.sqrt((2 * np.pi) ** num_features * det_cov)

        return np.exp(-0.5 * exponent) / norm_const

    def predict(self, X):
        """
        Dự đoán chỉ số của thành phần Gaussian chiếm ưu thế nhất cho mỗi điểm dữ liệu.
        """
        num_samples = X.shape[0]
        probs = np.zeros((num_samples, self.n_components))

        for k in range(self.n_components):
            probs[:, k] = self.weights[k] * self._multivariate_normal(X, self.means[k], self.covs[k])

        return np.argmax(probs, axis=1)

In [ ]:
import numpy as np

def PCA_reduce(data_matrix, num_components):
    """
    Giảm chiều dữ liệu bằng PCA (Principal Component Analysis) sử dụng NumPy.

    Parameters:
        data_matrix (ndarray): Ma trận dữ liệu đầu vào, kích thước (n_samples, n_features)
        num_components (int): Số chiều (thành phần chính) cần giữ lại

    Returns:
        projected_data (ndarray): Dữ liệu đã giảm chiều, kích thước (n_samples, num_components)
    """
    # Bước 1: Chuẩn hóa dữ liệu (trừ trung bình từng chiều)
    centered_data = data_matrix - np.mean(data_matrix, axis=0)

    # Bước 2: Tính ma trận hiệp phương sai
    cov_matrix = np.cov(centered_data, rowvar=False)

    # Bước 3: Phân rã trị riêng và vector riêng
    eig_values, eig_vectors = np.linalg.eigh(cov_matrix)

    # Bước 4: Sắp xếp các vector riêng theo trị riêng giảm dần
    sorted_eig_indices = np.argsort(eig_values)[::-1]
    eig_values = eig_values[sorted_eig_indices]
    eig_vectors = eig_vectors[:, sorted_eig_indices]

    # Bước 5: Chọn num_components vector riêng đầu tiên
    top_k_components = eig_vectors[:, :num_components]

    # Bước 6: Chiếu dữ liệu gốc lên không gian vector chính
    projected_data = centered_data @ top_k_components

    return projected_data


In [ ]:
def map_clusters_to_labels(true_labels, predicted_clusters, n_clusters):
    """
    Ánh xạ mỗi cụm được phân cụm sang nhãn thật, dựa trên nhãn chiếm đa số trong mỗi cụm.

    Parameters:
        true_labels (ndarray): Mảng nhãn thật, kích thước (N,)
        predicted_clusters (ndarray): Mảng nhãn cụm được gán, kích thước (N,)
        n_clusters (int): Số lượng cụm (cluster)

    Returns:
        mapped_labels (ndarray): Mảng nhãn dự đoán đã được ánh xạ về không gian nhãn thật
        cluster_to_label_map (dict): Từ điển ánh xạ cụm → nhãn thật chiếm đa số
    """
    cluster_to_label_map = {}

    for cluster_id in range(n_clusters):
        # Lấy chỉ số các điểm thuộc cụm hiện tại
        cluster_indices = np.where(predicted_clusters == cluster_id)[0]
        if len(cluster_indices) == 0:
            continue

        # Lấy các nhãn thật tương ứng với các điểm đó
        cluster_true_labels = true_labels[cluster_indices]

        # Tìm nhãn thật phổ biến nhất trong cụm
        labels_in_cluster, label_counts = np.unique(cluster_true_labels, return_counts=True)
        most_common_label = labels_in_cluster[np.argmax(label_counts)]

        # Gán ánh xạ cụm → nhãn
        cluster_to_label_map[cluster_id] = most_common_label

    # Tạo mảng nhãn đã được ánh xạ cho toàn bộ tập dự đoán
    mapped_labels = np.array([cluster_to_label_map[cluster] for cluster in predicted_clusters])

    return mapped_labels, cluster_to_label_map

In [ ]:
def evaluate(true_labels, predicted_labels):
    """
    Đánh giá độ chính xác theo từng lớp với các chỉ số: precision, recall và F1-score.

    Parameters:
        true_labels (ndarray): Nhãn thật của dữ liệu (ground truth), dạng (N,)
        predicted_labels (ndarray): Nhãn dự đoán, dạng (N,)

    Returns:
        metrics_per_label (dict): Từ điển chứa precision, recall, f1 cho từng nhãn.
    """
    unique_labels = np.unique(true_labels)
    metrics_per_label = {}

    for label in unique_labels:
        # Tính true positive, false positive, false negative cho nhãn hiện tại
        true_positives = np.sum((true_labels == label) & (predicted_labels == label))
        false_positives = np.sum((true_labels != label) & (predicted_labels == label))
        false_negatives = np.sum((true_labels == label) & (predicted_labels != label))

        # Tính precision, recall, f1 (có cộng 1e-12 để tránh chia cho 0)
        precision = true_positives / (true_positives + false_positives + 1e-12)
        recall = true_positives / (true_positives + false_negatives + 1e-12)
        f1_score = 2 * precision * recall / (precision + recall + 1e-12)

        # Lưu lại vào từ điển kết quả
        metrics_per_label[label] = {
            "precision": precision,
            "recall": recall,
            "f1": f1_score
        }

    return metrics_per_label

In [ ]:
X_train_reduced = PCA_reduce(X_Train, num_components=78)
X_test_reduced = PCA_reduce(X_Test, num_components=78)
GMM = GaussianMixtureModel(n_components=3, max_iter=200, tol=1e-3)
GMM.fit(X_train_reduced, verbose=True)

In [ ]:
y_test_cluster = GMM.predict(X_test_reduced)
y_test_pred, cluster_to_label = map_clusters_to_labels(y_Test, y_test_cluster, n_clusters=3)
scores = evaluate(y_Test, y_test_pred)
print("Đánh giá kết quả trên tập test:")
for label, metric in scores.items():
    print(f"Sentiment {label}: Precision={metric['precision']:.3f}, Recall={metric['recall']:.3f}, F1={metric['f1']:.3f}")

Đánh giá kết quả trên tập test:
Sentiment negative: Precision=0.510, Recall=0.421, F1=0.461
Sentiment neutral: Precision=0.000, Recall=0.000, F1=0.000
Sentiment positive: Precision=0.536, Recall=0.676, F1=0.598


In [ ]:
def count_labels(y):
    unique_labels, counts = np.unique(y, return_counts=True)
    for label, count in zip(unique_labels, counts):
        print(f"{label}: {count}")

count_labels(y_Test)

negative: 1409
neutral: 167
positive: 1590


In [ ]:
print(cluster_to_label)

{0: np.str_('positive'), 1: np.str_('negative'), 2: np.str_('positive')}


In [ ]:
def analyze_cluster_distribution(true_labels, predicted_clusters, n_clusters):
    """
    Phân tích phân bố nhãn thật (true labels) trong từng cụm được phân cụm.

    Parameters:
        true_labels (ndarray): Mảng nhãn thật (ground truth), kích thước (N,)
        predicted_clusters (ndarray): Mảng nhãn cụm đã gán (clustering output), kích thước (N,)
        n_clusters (int): Tổng số cụm (clusters) đã phân

    Returns:
        None (in kết quả trực tiếp ra màn hình)
    """
    print("📊 Phân bố nhãn thật trong từng cụm:")

    for cluster_id in range(n_clusters):
        # Lấy chỉ số các điểm thuộc cụm hiện tại
        cluster_indices = np.where(predicted_clusters == cluster_id)[0]

        if len(cluster_indices) == 0:
            print(f"\nCluster {cluster_id}: (trống)")
            continue

        # Lấy nhãn thật tương ứng với các điểm trong cụm
        cluster_true_labels = true_labels[cluster_indices]
        labels_in_cluster, label_counts = np.unique(cluster_true_labels, return_counts=True)

        print(f"\nCluster {cluster_id} ({len(cluster_indices)} điểm):")
        for label, count in zip(labels_in_cluster, label_counts):
            percentage = (count / len(cluster_indices)) * 100
            print(f"  - {label}: {count} mẫu ({percentage:.2f}%)")

analyze_cluster_distribution(y_Test, y_test_cluster, n_clusters=3)

📊 Phân bố nhãn thật trong từng cụm:

Cluster 0 (1 điểm):
  - positive: 1 (100.00%)

Cluster 1 (1162 điểm):
  - negative: 593 (51.03%)
  - neutral: 54 (4.65%)
  - positive: 515 (44.32%)

Cluster 2 (2003 điểm):
  - negative: 816 (40.74%)
  - neutral: 113 (5.64%)
  - positive: 1074 (53.62%)


### Nhận xét
-Trên tập test, mô hình GMM thể hiện chưa hiệu quả với nhãn negative khi chỉ phân cụm đúng khoảng một nửa số mẫu, phần còn lại bị nhầm sang các cụm khác. Đối với positive, kết quả có cải thiện nhẹ: khả năng nhận diện đúng cao hơn nhưng vẫn chưa thật sự chính xác. Đáng chú ý, với nhãn neutral, mô hình hoàn toàn không phát hiện được mẫu nào thuộc nhãn này.

-Dữ liệu test cho thấy số lượng mẫu neutral là cực kỳ ít. Khi ánh xạ cụm sang nhãn bằng cách chọn nhãn chiếm đa số trong mỗi cụm, có tới hai cụm bị gán là positive, trong khi không cụm nào đại diện cho neutral. Điều này được làm rõ hơn qua phân tích chi tiết phân bố nhãn trong từng cụm: các mẫu neutral không tập trung thành một cụm riêng mà bị rải rác và lẫn vào các cụm khác.

-Nguyên nhân chính là do sự mất cân bằng nhãn – neutral quá hiếm, khiến GMM, một mô hình không giám sát, không có lý do để hình thành cụm tương ứng. Ngoài ra, cũng có thể do biểu diễn đặc trưng từ PhoBERT chưa thể hiện tốt đặc điểm của nhãn neutral, hoặc quá trình giảm chiều đã làm mất thông tin quan trọng.

-Tóm lại, GMM hoạt động tương đối ổn với positive và negative, nhưng hoàn toàn thất bại trong việc nhận diện neutral, chủ yếu do dữ liệu không cân bằng và giới hạn trong biểu diễn đặc trưng.